<a href="https://colab.research.google.com/github/Mounish2023/fastaicourse/blob/main/RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration and loading

In [2]:
from fastai.text.all import *

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/test')]

In [5]:
path.ls()[1]

Path('/root/.fastai/data/imdb/train')

In [6]:
# (path/'/root/.fastai/data/imdb/imdb.vocab').open().read()

In [7]:
(path/'/root/.fastai/data/imdb/train').ls()

(#4) [Path('/root/.fastai/data/imdb/train/unsupBow.feat'),Path('/root/.fastai/data/imdb/train/labeledBow.feat'),Path('/root/.fastai/data/imdb/train/pos'),Path('/root/.fastai/data/imdb/train/neg')]

In [8]:
(path/'/root/.fastai/data/imdb/test').ls()

(#3) [Path('/root/.fastai/data/imdb/test/labeledBow.feat'),Path('/root/.fastai/data/imdb/test/pos'),Path('/root/.fastai/data/imdb/test/neg')]

In [9]:
(path/'/root/.fastai/data/imdb/README').open().read()

"Large Movie Review Dataset v1.0\n\nOverview\n\nThis dataset contains movie reviews along with their associated binary\nsentiment polarity labels. It is intended to serve as a benchmark for\nsentiment classification. This document outlines how the dataset was\ngathered, and how to use the files provided. \n\nDataset \n\nThe core dataset contains 50,000 reviews split evenly into 25k train\nand 25k test sets. The overall distribution of labels is balanced (25k\npos and 25k neg). We also include an additional 50,000 unlabeled\ndocuments for unsupervised learning. \n\nIn the entire collection, no more than 30 reviews are allowed for any\ngiven movie because reviews for the same movie tend to have correlated\nratings. Further, the train and test sets contain a disjoint set of\nmovies, so no significant performance is obtained by memorizing\nmovie-unique terms and their associated with observed labels.  In the\nlabeled train/test sets, a negative review has a score <= 4 out of 10,\nand a pos

In [10]:
files = get_text_files(path, folders=['train','test','unsup'])

In [11]:
txt = files[0].open().read()

# Tokenization and Numericalization

In [12]:
spacy = WordTokenizer()
toks = first(spacy([txt]))

In [13]:
# ??first

In [14]:
toks

(#167) ['I',"'m",'60','years','old',',','a','guitarist',',','(','lead','/','rhythm',')',',','and','over','the','last','forty'...]

In [15]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#184) ['xxbos','xxmaj','i',"'m",'60','years','old',',','a','guitarist',',','(','lead','/','rhythm',')',',','and','over','the','last','forty','years',',','xxmaj','i',"'ve",'been','in','four','bands'...]


In [16]:
# tkn.vocab

In [17]:
txts = L(o.open().read() for o in files[:2000])

In [18]:
# txts

In [19]:
txt

"I'm 60 years old, a guitarist, (lead/rhythm), and over the last forty years, I've been in four bands, it's all there, the fights, the foul-ups, the rotten food, the worse accommodation, always travelling, little or no money, and every one was drunk or high. But, the clubs, the fans, and the music, made it all worth it! Just like Strange Fruit! I'm too damn old for it now, and the arthritis in the hands and hips mean no more rocking, but for the length of that video, it all came back, and it was all there! The birds, the brawls, and the booze! And I was young again! It's just like Billy Connolly's voice over, God likes that 70's stuff! Rock On Forever!"

In [20]:
def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)

  return ' '.join((first(sp([txt]))[:40]))

In [21]:
sb = subword(1000)

In [22]:
sb

"▁I ' m ▁ 6 0 ▁years ▁old , ▁a ▁g u it ar ist , ▁( le ad / r h y th m ), ▁and ▁over ▁the ▁last ▁for ty ▁years , ▁I ' ve ▁been ▁in ▁f"

In [23]:
subword(200)

"▁I ' m ▁ 6 0 ▁ y e ar s ▁ ol d , ▁a ▁g u i t ar i s t , ▁ ( le a d / r h y th m ) , ▁and ▁"

In [24]:
subword(10000)

"▁I ' m ▁60 ▁years ▁old , ▁a ▁ guitar ist , ▁( le ad / r hy th m ), ▁and ▁over ▁the ▁last ▁for ty ▁years , ▁I ' ve ▁been ▁in ▁four ▁bands , ▁it ' s"

In [25]:
toks = tkn(txt)

In [26]:
toks

(#184) ['xxbos','xxmaj','i',"'m",'60','years','old',',','a','guitarist',',','(','lead','/','rhythm',')',',','and','over','the'...]

In [27]:
print(coll_repr(tkn(txt),31))

(#184) ['xxbos','xxmaj','i',"'m",'60','years','old',',','a','guitarist',',','(','lead','/','rhythm',')',',','and','over','the','last','forty','years',',','xxmaj','i',"'ve",'been','in','four','bands'...]


In [29]:
# ??map

In [30]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#184) ['xxbos','xxmaj','i',"'m",'60','years','old',',','a','guitarist',',','(','lead','/','rhythm',')',',','and','over','the'...]

you pass the list of tokens available to the Numericalize object during setup. this will create the vocabulary and also the numericals needed for it.

In [31]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2288) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [32]:
num.max_vocab,num.min_freq

(60000, 3)

In [33]:
nums = num(toks)[:20]; nums

TensorText([  2,   8,  19, 193, 949, 165, 170,  10,  13,   0,  10,  39, 585,
            103,   0,  36,  10,  12, 143,   9])

In [34]:
' '.join([num.vocab[o]  for o in nums])

"xxbos xxmaj i 'm 60 years old , a xxunk , ( lead / xxunk ) , and over the"

# Dataloaders

In [35]:
len(txts)

2000

In [36]:
toks200[0]

(#184) ['xxbos','xxmaj','i',"'m",'60','years','old',',','a','guitarist',',','(','lead','/','rhythm',')',',','and','over','the'...]

In [37]:
nums200 = toks200.map(num)

In [38]:
nums200[0]

TensorText([   2,    8,   19,  193,  949,  165,  170,   10,   13,    0,   10,
              39,  585,  103,    0,   36,   10,   12,  143,    9,  230,    0,
             165,   10,    8,   19,  152,  115,   17,  633,  586,   10,   18,
              23,   45,   55,   10,    9, 1680,   10,    9,    0,   25,    0,
              10,    9,    0, 1351,   10,    9, 1352,    0,   10,  242,    0,
              10,  155,   70,  126,  587,   10,   12,  283,   42,   26,    0,
              70,  526,   11,    8,   31,   10,    9,    0,   10,    9,  426,
              10,   12,    9,  243,   10,  112,   18,   45,  362,   18,   51,
               8,   68,   54,    8,  634,    8,    0,   51,    8,   19,  193,
              88,    0,  170,   30,   18,  167,   10,   12,    9,    0,   17,
               9,  950,   12,    0,  588,  126,   56,    0,   10,   31,   30,
               9, 1128,   14,   20,  527,   10,   18,   45,  292,  132,   10,
              12,   18,   26,   45,   55,   51,    8,    9,    0

In [39]:
dl = LMDataLoader(nums200)

In [40]:
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

**Exploring the dataloaders**

In [41]:
' '.join([num.vocab[o] for o in x[0]])

"xxbos xxmaj i 'm 60 years old , a xxunk , ( lead / xxunk ) , and over the last xxunk years , xxmaj i 've been in four bands , it 's all there , the fights , the xxunk - xxunk , the xxunk food , the worse xxunk , always xxunk , little or no money , and every one was xxunk or high . xxmaj but ,"

In [42]:
' '.join([num.vocab[o] for o in y[0]])

"xxmaj i 'm 60 years old , a xxunk , ( lead / xxunk ) , and over the last xxunk years , xxmaj i 've been in four bands , it 's all there , the fights , the xxunk - xxunk , the xxunk food , the worse xxunk , always xxunk , little or no money , and every one was xxunk or high . xxmaj but , the"

In [43]:
' '.join([num.vocab[o] for o in x[1]])

"play , and xxmaj bake xxunk begins to entertain them with some dancing , which ultimately helps xxunk the mood of the film , as well as bring happiness to the xxunk . \n\n xxmaj after xxunk to the couple 's xxunk stories , xxmaj sherry is trying to xxunk to xxmaj bake that she does not need his help with xxunk a job . xxmaj unfortunately xxmaj bake is unaware that"

In [44]:
' '.join([num.vocab[o] for o in y[1]])

", and xxmaj bake xxunk begins to entertain them with some dancing , which ultimately helps xxunk the mood of the film , as well as bring happiness to the xxunk . \n\n xxmaj after xxunk to the couple 's xxunk stories , xxmaj sherry is trying to xxunk to xxmaj bake that she does not need his help with xxunk a job . xxmaj unfortunately xxmaj bake is unaware that she"

In [45]:
dls = iter(dl)

In [46]:
x2,y2 = next(dls)
x2.shape, y2.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [47]:
' '.join([num.vocab[o] for o in x2[0]])

"xxbos xxmaj i 'm 60 years old , a xxunk , ( lead / xxunk ) , and over the last xxunk years , xxmaj i 've been in four bands , it 's all there , the fights , the xxunk - xxunk , the xxunk food , the worse xxunk , always xxunk , little or no money , and every one was xxunk or high . xxmaj but ,"

In [48]:
x3,y3 = next(dls)
x3.shape, y3.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [49]:
' '.join([num.vocab[o] for o in x3[0]])

"the xxunk , the fans , and the music , made it all worth it ! xxmaj just like xxmaj strange xxmaj xxunk ! xxmaj i 'm too xxunk old for it now , and the xxunk in the hands and xxunk mean no more xxunk , but for the length of that video , it all came back , and it was all there ! xxmaj the xxunk , the xxunk"

In [50]:
dl_list = list(dl)
x_last, y_last = dl_list[-1]

In [51]:
x_last.shape, y_last.shape

(torch.Size([64, 31]), torch.Size([64, 31]))

In [52]:
' '.join([num.vocab[o] for o in x_last[0]])

'playing coming from xxmaj bake and his xxunk when they missed the xxunk to xxunk on xxunk for the company . xxmaj the ladies are intrigued and ask to hear them'

In [53]:
x_lastb, y_lastb = dl_list[-2]

In [54]:
x_lastb.shape,y_lastb.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [55]:
' '.join([num.vocab[o] for o in x_lastb[0]])

'changed as another xxunk is xxunk into the film when a " party of big shots " is taking a tour of the ship . xxmaj this xxunk storyline is used to create a xxunk between the stories of the two main relationships of the film , as well as help xxunk xxunk and xxunk of xxmaj bake \'s overall good xxunk . xxmaj the party is xxunk by the xxunk music'

# ULMFit Approach

# Training a fine-tuned model on IMDB dataset

In [81]:
get_imdb = partial(get_text_files, folders=['train','test','unsup'])
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path=path,is_lm=True),
    get_items=get_imdb,
    splitter= RandomSplitter(0.1)
).dataloaders(source=path,path=path,bs=64,seq_len=80)

In [82]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj if you want to checkout a good xxmaj jason xxmaj scott xxmaj lee film , i recommend the following : \n\n xxmaj dragon : xxmaj the xxmaj bruce xxmaj lee xxmaj story \n\n xxmaj xxunk xxmaj nui \n\n "" timecop 2 : xxmaj the xxmaj berlin xxmaj decision "" is an awful film . xxmaj awful production values . xxmaj awful acting . xxmaj awful script . i would not recommend this film to be watched by anyone","xxmaj if you want to checkout a good xxmaj jason xxmaj scott xxmaj lee film , i recommend the following : \n\n xxmaj dragon : xxmaj the xxmaj bruce xxmaj lee xxmaj story \n\n xxmaj xxunk xxmaj nui \n\n "" timecop 2 : xxmaj the xxmaj berlin xxmaj decision "" is an awful film . xxmaj awful production values . xxmaj awful acting . xxmaj awful script . i would not recommend this film to be watched by anyone who"
1,kill him . xxmaj chills . i also love the frogs falling from the sky scene and i love xxmaj scully 's need to explain it somehow and showing up to tell xxmaj mulder about the tornadoes that could have picked them up and dropped them down on top of them . i love the pig coming alive and the girl totally freaking out and screaming bloody murder . xxmaj that must have been an interesting class to be in,him . xxmaj chills . i also love the frogs falling from the sky scene and i love xxmaj scully 's need to explain it somehow and showing up to tell xxmaj mulder about the tornadoes that could have picked them up and dropped them down on top of them . i love the pig coming alive and the girl totally freaking out and screaming bloody murder . xxmaj that must have been an interesting class to be in you


In [58]:
learn = language_model_learner(
    dls_lm,AWD_LSTM,drop_mult=0.3,
    metrics=[accuracy,Perplexity()]
).to_fp16()

In [59]:
learn.fit_one_cycle(1,2e-2)

/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.994165,3.904962,0.300679,49.648209,20:25


In [60]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [61]:
!ls /root/.fastai/data/imdb/models/


1epoch.pth


In [62]:
!cp /root/.fastai/data/imdb/models/1epoch.pth /content/


learn.load() was not working to load the model. after PyTorch > 2.6, the pytorch changed the weights_only parameter to True as a default for security reasons. we have to make it false to get the parameters. and this cannot be done using fastai. so have to load it using torch.load(path, weights_only=False). the path has to be full path to the model saved. and the result is a dict not the model_state alone. and to load the model we have reinstantiate the model with dataloaders and architectures. and the model state can be loaded with learn2.model.load_state_dict(state=model_state)

In [ ]:
# learn = torch.load('1epoch', weights_only=True)

In [ ]:
# learn =learn.load('1epoch')

In [68]:
from google.colab import drive
drive.mount('/content/drive')

!cp /root/.fastai/data/imdb/models/1epoch.pth /content/drive/MyDrive/fastai/


Mounted at /content/drive


In [83]:
learn2 = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics = [accuracy, Perplexity()]
).to_fp16()

In [84]:
import torch
checkpoint =torch.load('/root/.fastai/data/imdb/models/1epoch.pth', weights_only=False)
model_state = checkpoint['model']
learn2.model.load_state_dict(model_state)


<All keys matched successfully>

In [85]:
total_params = sum(p.numel() for p in learn2.parameters())
total_params

44336488

In [80]:
model = learn2
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()
size_all_mb = (param_size + buffer_size) / 1024**2
print(f"Approx. model size in memory: {size_all_mb:.2f} MB")


Approx. model size in memory: 169.13 MB


In [1]:
learn2.unfreeze()
learn2.fit_one_cycle(2,2e-3)

NameError: name 'learn2' is not defined